# Sudoku solver using Symulated Annealing
Agnieszka Dutka

Go to:
[SA main algorithm](#sa)
[Testing sudokus](#testing)

In [85]:
import numpy as np
from random import random, sample
from math import exp
import matplotlib.pyplot as plt
import matplotlib.cm as cm

### Useful structures

In [87]:
class Sudoku:
    def __init__(self, matrix=None, fromSudoku=None):
        if fromSudoku is None:
            self.matrix=matrix
            self.setUnknowns()
        else:
            self.matrix = np.copy(fromSudoku.matrix)
            self.unknowns = fromSudoku.unknowns
                    
    def setUnknowns(self): # initialize empty fields wth sth
        self.unknowns = []
        nrToFill = []
        for r in range(9):
            nrs = [0]*10
            for c in range(9):
                if self[r, c] < 0:
                    self.unknowns.append([r, c])
                else:
                    nrs[self[r, c]] = 1
            for c in range(1, 10):
                if nrs[c] == 0:
                    nrToFill.append(c)
        for idx, nr in enumerate(nrToFill):
            r, c = self.unknowns[idx]
            self[r, c] = nr
                    
    def __setitem__(self, index, value):
        self.matrix[index] = value

    def __getitem__(self, index):
        return self.matrix[index]
        
    def __repr__(self):
        return str(self.matrix)

In [92]:
class OptimState:
    def __init__(self, x0, cost, T0, alpha, iterNr=0):
        self.val = x0
        self.cost = cost
        self.T0 = T0
        self.alpha = alpha
        self.iter = iterNr

    def __repr__(self):
        adj = "" if self.cost ==0 else "approx "
        return f"T0: {self.T0}\nalpha:{self.alpha}\n{adj}solution:\n{self.val}\ncost: {self.cost}\niter: {self.iter}"

In [93]:
def readSudoku(filename):
    """ - each line must represent one row, 
        - empty place marked as any char except whitespace 
        - see sudokus folder for examples """
    f = open(filename, "r", encoding='utf-8')
    text = f.readlines()
    mx = np.zeros((9, 9), dtype=int)
    lnr=0
    for line in text:
        line.replace(" ", "")
        for i in range(9):
            mx[lnr][i] =  -1 if (line[i] < '1' or line[i] > '9') else ord(line[i])-48
        lnr+=1
    return Sudoku(mx)


<a id='sa'></a>
## SA main algorithm

In [94]:
def SA(costFunc, x, nextSudoku, T0=100., show=0, maxIter=20000, alpha=0.99):
    alpha = alpha
    state = OptimState(x, costFunc(x), T0, alpha)
    costs = [state.cost]
    T = T0
    for i in range(maxIter):
        
        if state.cost == 0:
            print("ideal solution found")
            return state
        
        newX = nextSudoku(state.val)
        newState = OptimState(newX, costFunc(newX), T0, alpha, i)
        
        # decide whether change state
        delta = newState.cost - state.cost
        if delta <= 0:
            state = newState
        else:
            P = exp(-1*delta / T)
            if (show>0 and i % show == 0):
                print(P, " on iter ", i)
                print("T = ", T)
            if random() <= P:
                state = newState

        T = T*alpha
        costs.append(state.cost)
        if (show>0 and i %show == 0): #i % show ==0
            print(state.val)
            print(state.cost)
            
    return state


### cost function

In [95]:
def costFunc(sudoku: Sudoku):
    cost = 0
    for r in range(9):
        nrs = [0]*10
        for c in range(9):
            nrs[sudoku[r,c]] += 1
        for idx in range(1, 10):
            if nrs[idx] > 0:
                nrs[idx] -= 1
        cost += np.sum(nrs)
    for c in range(9):
        nrs = [0]*10
        for r in range(9):
            nrs[sudoku[r,c]] += 1
        for idx in range(1, 10):
            if nrs[idx] > 0:
                nrs[idx] -= 1
        cost += np.sum(nrs)
    for i in range(0, 9, 3):
        for j in range(0, 9, 3):
            nrs = [0]*10
            for r in range(3):
                for c in range(3):
                    nrs[sudoku[i+r,j+c]] += 1
            for idx in range(1, 10):
                if nrs[idx] > 0:
                    nrs[idx] -= 1
            cost += np.sum(nrs)
    return cost

### next state function

In [96]:
def nextStateRnd(sudoku: Sudoku):
    su = sudoku.unknowns
    idxs =  sample([x for x in range(len(su))], 2)
    coord1, coord2 = su[idxs[0]], su[idxs[1]]
    r1, c1 = coord1
    r2, c2 = coord2
    res = Sudoku(fromSudoku=sudoku)
    res[r1, c1], res[r2, c2] = sudoku[r2, c2], sudoku[r1, c1]
    return res

<a id='testing'></a>
### Testing

In [98]:
sudoku =readSudoku("sudokus/ex1Ez.txt")
print("sudoku:\n",sudoku)
print("unknowns: ",len(sudoku.unknowns))
print("initial cost: ",costFunc(sudoku), "\n")

res = SA(costFunc, sudoku, nextStateRnd, show=0, maxIter=30000, T0=320)
print(res)

sudoku:
 [[1 2 3 4 5 6 7 8 9]
 [4 5 7 1 8 9 2 3 6]
 [6 8 9 2 3 7 1 4 5]
 [1 8 5 3 6 2 9 7 4]
 [2 7 4 1 9 8 6 5 3]
 [3 9 6 5 7 4 8 1 2]
 [2 4 5 6 1 8 3 9 7]
 [7 6 1 3 4 9 5 2 8]
 [9 3 8 7 2 5 1 6 4]]
unknowns:  18
initial cost:  10 

ideal solution found
T0: 320
alpha:0.99
solution:
[[1 2 3 4 5 6 7 8 9]
 [4 5 7 1 8 9 2 3 6]
 [6 8 9 2 3 7 1 4 5]
 [8 1 5 3 6 2 9 7 4]
 [2 7 4 8 9 1 6 5 3]
 [3 9 6 5 7 4 8 1 2]
 [5 4 2 6 1 8 3 9 7]
 [7 6 1 9 4 3 5 2 8]
 [9 3 8 7 2 5 4 6 1]]
cost: 0
iter: 817
